In [1]:
import math as m
import numpy as np
import matplotlib.pyplot as plt
from numpy import random

In [60]:
number_detectors = 2
number_gw_polarizations = 2
number_gw_modes = 4
number_source_angles = 3
hanford_detector_angles = [(46+27/60+18.528/3600)*np.pi/180, (240+35/60+32.4343/3600)*np.pi/180,np.pi/2+125.9994*np.pi/180]
livingston_detector_angles = [(30+33/60+46.4196/3600)*np.pi/180, (269+13/60+32.7346/3600)*np.pi/180,np.pi/2+197.7165*np.pi/180]
ligo_detector_sampling_rate = 16384
earth_radius = 6371000
speed_light = 299792458
maximum_hanford_livingston_time_delay = 0.010002567302556083
weighting_power = 2

NOTE: all angles are in radians and all dimension-ful quantities are in SI units (meters, seconds, kilograms, and combinations thereof) unless explicitly indicated otherwise.

In [61]:
def transform_2_0_tensor(matrix, change_basis_matrix) : 
    contravariant_transformation_matrix = np.linalg.inv(change_basis_matrix)
    partial_transformation = np.einsum('ki,kl->il', contravariant_transformation_matrix, matrix)
    return np.einsum('lj,il->ij', contravariant_transformation_matrix, partial_transformation)

def transform_1_1_tensor(matrix, change_basis_matrix) : 
    contravariant_transformation_matrix = np.linalg.inv(change_basis_matrix)
    partial_transformation = np.einsum('ik,kl->il', change_basis_matrix, matrix)
    return np.einsum('lj,il->ij', contravariant_transformation_matrix, partial_transformation)

def transform_0_2_tensor(matrix, change_basis_matrix) :
    partial_transformation = np.einsum('ik,kl->il', change_basis_matrix, matrix)
    return np.einsum('jl,il->ij', change_basis_matrix, partial_transformation)

These functions take an arbitrary matrix -- a NumPy array -- and the change-of-basis matrix -- another NumPy array -- that effects a change-of-basis -- i.e., transforms the basis vectors -- from the coordinate system in which the matrix is currently expressed to the coordinate system in which you would like the matrix to be expressed. The first function is for (2,0) tensors (i.e., two contravariant/upper indices), the second is for (1,1) tensors (true matrices), and the third is for (0,2) tensors (two covariant/lower indices).

In [62]:
def source_vector_from_angles(angles) :
    [first, second, third] = angles
    initial_source_vector = np.array([m.cos(first)*m.cos(second),m.cos(first)*m.sin(second),m.sin(first)])
    return initial_source_vector

This function takes a list with three angles in it -- either the declination, right ascension, and polarization angles of a gravitational wave source, or the latitute, longitude, and orientation  a gravitational wave detector -- and returns a 1D NumPy array representing the unit-length vector from the center  the Earth to the source/detector. Note that only the first two angles are actually needed to compute the unit vector.

In [63]:
def change_basis_gw_to_ec(source_angles) :
    [declination,right_ascension,polarization] = source_angles
    initial_source_vector = source_vector_from_angles(source_angles)
    initial_gw_z_vector_earth_centered = -1*initial_source_vector
    initial_gw_y_vector_earth_centered = np.array([-1*m.sin(declination)*m.cos(right_ascension),-1*m.sin(declination)*m.sin(right_ascension),m.cos(declination)])
    initial_gw_x_vector_earth_centered = np.cross(initial_gw_z_vector_earth_centered,initial_gw_y_vector_earth_centered)
    transpose_gw_vecs_ec = np.array([initial_gw_x_vector_earth_centered,initial_gw_y_vector_earth_centered,initial_gw_z_vector_earth_centered])
    initial_gw_vecs_ec = np.transpose(transpose_gw_vecs_ec)
    polarization_rotation_matrix = np.array([[m.cos(polarization),-1*m.sin(polarization),0],[m.sin(polarization),m.cos(polarization),0],[0,0,1]])
    contravariant_transformation_matrix = np.matmul(polarization_rotation_matrix,initial_gw_vecs_ec)
    change_basis_matrix = np.linalg.inv(contravariant_transformation_matrix)
    return change_basis_matrix

This function takes a list with the declination, right ascension, and polarization angles of a gravitational wave source in the Earth-centered coordinate system and returns a NumPy array that effects the change-of-basis (covariant transformation matrix) from the gravitational wave frame to the Earth-centered frame. The inverse of this matrix inverse effects the change-of-basis from the Earth-centered frame to the gravitational wave frame, and is also the contravariant transformation matrix (i.e., changes the components  vectors) from the gravitational wave frame to the Earth-centered frame.

In [64]:
def gravitational_wave_ec_frame(source_angles,tt_amplitudes) :
    [hplus,hcross] = tt_amplitudes
    gwtt = np.array([[hplus,hcross,0],[hcross,-hplus,0],[0,0,0]])
    transformation = change_basis_gw_to_ec(source_angles)
    return transform_0_2_tensor(gwtt,transformation)

This function takes two lists -- the first containing the declination, right ascension, and polarization angles of the source, the second containing the "plus" and "cross" strain amplitudes of the gravitational wave in the transverse, traceless ("TT") gauge of the gravitational wave frame -- and returns a NumPy array characterizing the gravitational wave's strain amplitudes in the Earth-centered frame. Note that the strain tensor is a (0-2) tensor.

In [65]:
def change_basis_detector_to_ec(detector_angles) :
    [latitude,longitude,orientation] = detector_angles
    initial_detector_z_vector_earth_centered = source_vector_from_angles(detector_angles)
    initial_detector_x_vector_earth_centered = np.array([-1*m.sin(longitude),m.cos(longitude),0])
    initial_detector_y_vector_earth_centered = np.cross(initial_detector_z_vector_earth_centered,initial_detector_x_vector_earth_centered)
    transpose_detector_vecs_ec = np.array([initial_detector_x_vector_earth_centered,initial_detector_y_vector_earth_centered,initial_detector_z_vector_earth_centered])
    initial_detector_vecs_ec = np.transpose(transpose_detector_vecs_ec)
    orientation_rotation_matrix = np.array([[m.cos(orientation),-1*m.sin(orientation),0],[m.sin(orientation),m.cos(orientation),0],[0,0,1]])
    contravariant_transformation_matrix = np.matmul(orientation_rotation_matrix,initial_detector_vecs_ec)
    change_basis_matrix = np.linalg.inv(contravariant_transformation_matrix)
    return change_basis_matrix

This function takes a list containing the latitude, longitude, and orientation  a gravitational wave detector and returns a NumPy array that effects the change-of-basis (covariant transformation matrix) from the detector frame to the Earth-centered frame.

In [66]:
def detector_response(detector_angles, source_angles, tt_amplitudes) :
    detector_response_tensor_detector_frame = np.array([[1/2,0,0],[0,-1/2,0],[0,0,0]])
    transform_detector_to_ec = change_basis_detector_to_ec(detector_angles)
    detector_response_tensor_earth_centered = transform_2_0_tensor(detector_response_tensor_detector_frame,transform_detector_to_ec)
    gw_earth_centered = gravitational_wave_ec_frame(source_angles,tt_amplitudes)
    detector_response = np.tensordot(gw_earth_centered,detector_response_tensor_earth_centered)
    return detector_response

This function takes three lists -- the first containing the latitude, longitude, and orientation angles of a gravitational wave detector, the second containing the declination, right ascension, and polarization angles of the source, and the third containing the "plus" and "cross" strain amplitudes of the gravitational wave in the transverse, traceless ("TT") gauge -- and returns a scalar representing the strain measured by the gravitational wave detector. Note that the detector response tensor is a (2-0) tensor.

In [67]:
def beam_pattern_response_functions(detector_angles,source_angles) :
    detector_response_tensor_detector_frame = np.array([[1/2,0,0],[0,-1/2,0],[0,0,0]])
    transform_detector_ec = change_basis_detector_to_ec(detector_angles)
    detector_response_tensor_earth_centered = transform_2_0_tensor(detector_response_tensor_detector_frame,transform_detector_ec)
    transform_gw_ec = change_basis_gw_to_ec(source_angles)
    transform_ec_gw = np.linalg.inv(transform_gw_ec)
    detector_response_tensor_gw_frame = transform_2_0_tensor(detector_response_tensor_earth_centered,transform_ec_gw)
    fplus = detector_response_tensor_gw_frame[0,0]-detector_response_tensor_gw_frame[1,1]
    fcross = detector_response_tensor_gw_frame[0,1]+detector_response_tensor_gw_frame[1,0]
    return [fplus, fcross]

This function takes two lists -- the first containing the latitude, longitude, and orientation angles of a gravitational wave detector, and the second containing the declination, right ascensions, and polarization angles of a gravitational wave source -- and returns a list with the beam pattern response functions $F_+$ and $F_\times$ of the detector for that source.

In [68]:
def time_delay_hanford_to_livingston(source_angles) :
    hanford_z_vector_earth_centered = source_vector_from_angles(hanford_detector_angles)
    livingston_z_vector_earth_centered = source_vector_from_angles(livingston_detector_angles)
    position_vector_hanford_to_livingston = earth_radius * (livingston_z_vector_earth_centered - hanford_z_vector_earth_centered)
    gw_source_vector = source_vector_from_angles(source_angles)
    gw_z_vector_earth_centered = -1*gw_source_vector
    return 1/speed_light*(np.dot(gw_z_vector_earth_centered,position_vector_hanford_to_livingston))

This function take a list of the declination, right ascension, and polarization angles of a gravitational wave source and returns the time delay between when the signal will arrive at the Hanford detector and when it will arrive at the Livingston detector. Negative values indicate that the signal arrives at the Livingston detector first.

In [69]:
def generate_network_time_array(signal_lifetime, detector_sampling_rate, maximum_time_delay) :
    time_sample_width = round((signal_lifetime + maximum_time_delay)*detector_sampling_rate,0)  
    all_times = 1/detector_sampling_rate*(np.arange(-time_sample_width,time_sample_width,1))
    return all_times

This function takes a gravitational wave signal lifetime, a detector sampling rate, and the maximum possible time delay between the detectors in a network, and returns a NumPy array with absolute detector strain response times appropriate for all the detectors in a network. Note that all responses times are actual sampled times, assuming correct time synchorization between sites.

In [70]:
def generate_oscillatory_terms(signal_lifetime, signal_frequency, time_array, time_delay) :
    number_time_samples = time_array.size
    signal_q_value = m.sqrt(m.log(2))/signal_lifetime
    oscillatory_terms = np.empty((number_time_samples,number_gw_modes))
    for this_time_sample in range(number_time_samples) :
        this_time = time_array[this_time_sample]
        this_gaussian_term = m.exp(-1*signal_q_value**2*(this_time - time_delay)**2)
        this_cos_term = m.cos(2*np.pi*signal_frequency*(this_time - time_delay))
        this_sin_term = m.sin(2*np.pi*signal_frequency*(this_time - time_delay))
        this_cos_gauss_term = this_gaussian_term*this_cos_term
        this_sin_gauss_term = this_gaussian_term*this_sin_term
        these_oscillations = np.array([this_cos_gauss_term,this_sin_gauss_term,this_cos_gauss_term,this_sin_gauss_term])
        oscillatory_terms[this_time_sample] = these_oscillations
    return oscillatory_terms

This function takes the lifetime and frequency of a gravitional wave, a NumPy array with the detector strain response times of a gravitational wave detector network, and the time delay between when the gravitational wave arrives at the specific detector where the terms are being evaluated compared to when it arrived at a fixed reference detector (Hanford, in this code), and returns a NumPy array with the appropriate sine-Gaussian gravitational wave strain amplitudes for the detector at each network time. Note that the order of the output array is 1) time sample 2) $[ A_+, B_+, A_\times, B_\times]$.

In [71]:
def generate_model_angles_array(number_angular_samples) : 
    model_angles_array = np.empty((number_angular_samples,number_source_angles))
    for this_angle_set in range(number_angular_samples) :
        for this_source_angle in range(number_source_angles) :
            if this_source_angle == 0 :
                model_angles_array[this_angle_set,this_source_angle] = (np.random.rand(1) - 1/2)*np.pi
            else :
                model_angles_array[this_angle_set,this_source_angle] = np.random.rand(1)*2*np.pi
    return model_angles_array

This functions takes the number of desired model angle sets $[\delta,\phi,\psi]$ and returns a NumPy array with that many randomized angle sets. Note that the first angle in each set is bewteen $-\frac{\pi}{2}$ and $\frac{\pi}{2}$, while all subsequent angles are between $0$ and $2 \pi$. 

In [72]:
def generate_model_amplitudes_array(number_amplitude_combinations, gw_max_amps) :
    model_amplitudes_array = np.empty((number_amplitude_combinations,number_gw_modes))
    for this_amplitude_combination in range(number_amplitude_combinations) :
        new_amplitudes = np.random.rand(number_gw_modes)*gw_max_amps
        model_amplitudes_array[this_amplitude_combination] = new_amplitudes 
    return model_amplitudes_array

This function takes the number of desired model amplitude combinations $[A_+, B_+, A_\times, B_\times]$ and the maximum allowed value for any amplitude and returns a NumPy array with the desired amplitude combinations.

In [73]:
def generate_model_detector_responses(signal_frequency,signal_lifetime,detector_sampling_rate,gw_max_amps,number_amplitude_combinations,number_angular_samples) :
    time_array = generate_network_time_array(signal_lifetime,detector_sampling_rate,maximum_hanford_livingston_time_delay)
    number_time_samples = time_array.size
    hanford_oscillatory_terms = generate_oscillatory_terms(signal_lifetime,signal_frequency,time_array,0)
    model_amplitudes_array = generate_model_amplitudes_array(number_amplitude_combinations, gw_max_amps)
    model_angles_array = generate_model_angles_array(number_angular_samples)
    model_detector_response_array = np.empty((number_angular_samples,number_amplitude_combinations,number_time_samples,number_detectors))
    for this_angle_set in range(number_angular_samples) :
        these_angles = model_angles_array[this_angle_set]
        [fplus_hanford,fcross_hanford] = beam_pattern_response_functions(hanford_detector_angles,these_angles)
        [fplus_livingston,fcross_livingston] = beam_pattern_response_functions(livingston_detector_angles,these_angles)
        hanford_livingston_time_delay = time_delay_hanford_to_livingston(these_angles)
        livingston_oscillatory_terms = generate_oscillatory_terms(signal_lifetime,signal_frequency,time_array,hanford_livingston_time_delay)
        for this_amplitude_combination in range(number_amplitude_combinations) :
            these_amplitudes = model_amplitudes_array[this_amplitude_combination]
            for this_sample_time in range(number_time_samples) :
                model_detector_response_array[this_angle_set,this_amplitude_combination,this_sample_time,0] = np.dot(these_amplitudes,hanford_oscillatory_terms[this_sample_time]*[fplus_hanford,fplus_hanford,fcross_hanford,fcross_hanford])
                model_detector_response_array[this_angle_set,this_amplitude_combination,this_sample_time,1] = np.dot(these_amplitudes,livingston_oscillatory_terms[this_sample_time]*[fplus_livingston,fplus_livingston,fcross_livingston,fcross_livingston])
    return [model_detector_response_array,model_angles_array]

This function takes three source/detector parameters -- the frequency and lifetime (here defined as the time required to drop to one-half of maximum amplitude) of (one monochromatic Fourier mode of) a gravitational wave, and the sampling rate of the detectors -- and three model parameters -- the maximum model graviational wave amplitude to consider and the number of amplitude and angle combinations to generate -- and returns a pair of NumPy arrays that give the expected response of each gravitational wave detector at each time for each amplitude and angle combination -- indexed by 1) angle combination 2) amplitude combination 3) time sample and 4) detector -- and the array of angle combinations referenced by the detector response array.

In [74]:
def generate_noise_array(max_noise_amp,number_time_samples) :
    noise_array = np.random.rand(number_time_samples)*max_noise_amp
    return noise_array

This function takes a maximum noise amplitude and a number of time samples and returns random (non-Gaussian) noise between zero and the appropriate maximum for each time sample.

In [75]:
def generate_real_detector_responses(signal_frequency,signal_lifetime,detector_sampling_rate,gw_max_amps,number_amplitude_combinations,number_angular_samples,max_noise_amp) :
    time_array = generate_network_time_array(signal_lifetime,detector_sampling_rate,maximum_hanford_livingston_time_delay)
    number_time_samples = time_array.size
    real_amplitudes = generate_model_amplitudes_array(1, gw_max_amps)[0]
    real_angles = generate_model_angles_array(1)[0]
    [fplus_hanford,fcross_hanford] = beam_pattern_response_functions(hanford_detector_angles,real_angles)
    [fplus_livingston,fcross_livingston] = beam_pattern_response_functions(livingston_detector_angles,real_angles)
    hanford_livingston_time_delay = time_delay_hanford_to_livingston(real_angles)
    hanford_oscillatory_terms = generate_oscillatory_terms(signal_lifetime,signal_frequency,time_array,0)
    livingston_oscillatory_terms = generate_oscillatory_terms(signal_lifetime,signal_frequency,time_array,hanford_livingston_time_delay)
    small_detector_response_array = np.empty((number_time_samples,number_detectors))
    hanford_noise_array = generate_noise_array(max_noise_amp,number_time_samples)
    livingston_noise_array = generate_noise_array(max_noise_amp,number_time_samples)
    for this_sample_time in range(number_time_samples) :
        small_detector_response_array[this_sample_time,0] = np.dot(real_amplitudes,hanford_oscillatory_terms[this_sample_time]*[fplus_hanford,fplus_hanford,fcross_hanford,fcross_hanford]) + hanford_noise_array[this_sample_time]
        small_detector_response_array[this_sample_time,1] = np.dot(real_amplitudes,livingston_oscillatory_terms[this_sample_time]*[fplus_livingston,fplus_livingston,fcross_livingston,fcross_livingston]) + livingston_noise_array[this_sample_time]
    real_angles_array = np.empty((number_angular_samples,number_source_angles))
    real_detector_response_array = np.empty((number_angular_samples,number_amplitude_combinations,number_time_samples,number_detectors))
    for this_angle_set in range(number_angular_samples) :
        real_angles_array[this_angle_set] = real_angles
        for this_amplitude_combination in range(number_amplitude_combinations) :
            real_detector_response_array[this_angle_set,this_amplitude_combination] = small_detector_response_array
    return [real_detector_response_array,real_angles_array]

This function takes three source/detector parameters -- the frequency and lifetime (here defined as the time required to drop to one-half of maximum amplitude) of (one monochromatic Fourier mode of) a gravitational wave, and the sampling rate of the detectors -- and three model parameters -- the maximum model graviational wave amplitude to consider and the number of amplitude and angle combinations in the model -- and returns a pair of NumPy arrays that give the "true" (simulated) response of each gravitational wave detector at each time for each amplitude and angle combination -- indexed by 1) angle combination 2) amplitude combination 3) time sample and 4) detector -- and the "true" simulated source angles for each angle combination referenced by the detector response array. Note that there is a great deal of repeated information in these arrays, but they are intentionally size-matched with the outputs of the function "generate_model_detector_responses" to ease later array computations.

In [76]:
def get_best_fit_angles_deltas(real_detector_responses,real_angles_array,model_detector_responses,model_angles_array) :
    real_model_angle_deltas = np.absolute(real_angles_array - model_angles_array)
    summed_real_model_angle_deltas = np.sum(real_model_angle_deltas,-1)
    minimum_summed_angle_delta = np.min(summed_real_model_angle_deltas)
    position_minimum_angles_delta = np.where(summed_real_model_angle_deltas == minimum_summed_angle_delta)
    angles_minimum_angles_delta = model_angles_array[position_minimum_angles_delta[0]]
    real_minimum_angles_deltas = np.absolute(real_angles_array[0] - angles_minimum_angles_delta)
    sum_real_minimum_angle_deltas = np.sum(real_minimum_angles_deltas)
    
    real_model_response_deltas = np.absolute(real_detector_responses - model_detector_responses)
    summed_real_model_response_deltas = np.sum(real_model_response_deltas,axis=(-1,-2))
    minimum_summed_response_delta = np.min(summed_real_model_response_deltas)
    position_minimum_response_delta = np.where(summed_real_model_response_deltas == minimum_summed_response_delta)
    angles_minimum_response_delta = model_angles_array[position_minimum_response_delta[0]]
    real_minimum_response_angle_deltas = np.absolute(real_angles_array[0] - angles_minimum_response_delta)
    sum_real_minimum_response_angle_deltas = np.sum(real_minimum_response_angle_deltas)
    
    offset_matrix = np.ones(summed_real_model_response_deltas.shape)
    fractional_summed_real_model_response_deltas = 1/minimum_summed_response_delta * summed_real_model_response_deltas
    weighted_summed_real_model_response_deltas = np.exp(offset_matrix - fractional_summed_real_model_response_deltas**weighting_power)
    summed_weighted_summed_real_model_response_deltas = np.sum(weighted_summed_real_model_response_deltas,axis=-1)
    maximum_summed_weighted_response_delta = np.max(summed_weighted_summed_real_model_response_deltas)
    position_maximum_summed_weighted_response_delta = np.where(summed_weighted_summed_real_model_response_deltas == maximum_summed_weighted_response_delta)
    angles_maximum_summed_weighted_response_delta = model_angles_array[position_maximum_summed_weighted_response_delta[0]]
    real_maximum_weighted_response_angle_deltas = np.absolute(real_angles_array[0] - angles_maximum_summed_weighted_response_delta)
    sum_real_maximum_weighted_response_angle_deltas = np.sum(real_maximum_weighted_response_angle_deltas)
    
    return [sum_real_minimum_angle_deltas,sum_real_minimum_response_angle_deltas,sum_real_maximum_weighted_response_angle_deltas]

This function takes NumPy arrays containing the "real" (simulated) detector responses and source angles and the model detector responses and source angles and returns the sum of the absolute values of the differences between the "real" (simulated) angles and 1) the angles in model_angles_array that are closest to the "real" angles (i.e., the best the fitting procedure could have done given the model angles it used) 2) the angles produced by finding the single best time-and-site summed model detector response and using the angles that produce it and 3) the angles produced by weighting the time-and-site summed detector responses, summing them over all amplitude combinations, and using the angles produced by maximizing that sum. 

In [77]:
gw_frequency = 100
gw_lifetime = 0.03
detector_sampling_rate = 10000
gw_max_amps = 1
max_noise_amp = 0
number_angular_samples = 5
number_amplitude_combinations = 10
[model_detector_responses,model_angles_array] = generate_model_detector_responses(gw_frequency,gw_lifetime,detector_sampling_rate,gw_max_amps,number_amplitude_combinations,number_angular_samples)
[real_detector_responses,real_angles_array] = generate_real_detector_responses(gw_frequency,gw_lifetime,detector_sampling_rate,gw_max_amps,number_amplitude_combinations,number_angular_samples,max_noise_amp)
best_fit_data = get_best_fit_angles_deltas(real_detector_responses,real_angles_array,model_detector_responses,model_angles_array)
print(best_fit_data)

[4.633124786514383, 7.920558338010169, 7.920558338010169]
